In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
%time

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [2]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import opendatasets as od
import pandas
  
od.download(
    "https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification/download")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nandarlma
Your Kaggle Key: ··········


100%|██████████| 82.0M/82.0M [00:00<00:00, 91.5MB/s]


In [4]:
import glob, os
import cv2 as cv
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout,BatchNormalization
from keras.models import Sequential

In [5]:
base_path = '/content/garbage-classification/garbage classification/Garbage classification'

img_list = glob.glob(os.path.join(base_path, '*/*.jpg'))

print(len(img_list))

2527


In [6]:
image_size = 512

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

validation_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 2024 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [20]:
model2 = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', input_shape=(image_size, image_size, 3)),
    MaxPooling2D(pool_size=2),

    Conv2D(64, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(128, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),

    Dense(64, activation='relu'),

    Dense(6, activation='softmax')
])

model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 510, 510, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 255, 255, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 126, 126, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 62, 62, 128)     

In [21]:
from tensorflow import keras 
model2.compile(loss = 'categorical_crossentropy', optimizer= 'adam',metrics=["accuracy",keras.metrics.Precision(),keras.metrics.Recall()])

In [22]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model2.fit_generator(train_generator,
                    epochs=20,
                    validation_data=validation_generator,
                    verbose = 1,
                    callbacks=[early_stopping])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/20
127/127 [==============================] - 111s 867ms/step - loss: 2.0939 - accuracy: 0.2080 - precision_4: 0.1964 - recall_4: 0.0054 - val_loss: 1.7440 - val_accuracy: 0.2351 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 2/20
127/127 [==============================] - 119s 934ms/step - loss: 1.7289 - accuracy: 0.2456 - precision_4: 0.2400 - recall_4: 0.0030 - val_loss: 1.6847 - val_accuracy: 0.2908 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 3/20
127/127 [==============================] - 109s 853ms/step - loss: 1.6503 - accuracy: 0.3167 - precision_4: 0.5326 - recall_4: 0.0242 - val_loss: 1.6832 - val_accuracy: 0.2908 - val_precision_4: 0.4000 - val_recall_4: 0.0159
Epoch 4/20
127/127 [==============================] - 107s 841ms/step - loss: 1.5312 - accuracy: 0.3780 - precision_4: 0.5891 - recall_4: 0.0800 - val_loss: 1.4375 - val_accuracy: 0.3984 - val_precision_4: 0.6981 - val_recall_4: 0.1474
Epoch 5/20
127/127 [====================

In [23]:
model2.fit_generator(train_generator,
                    epochs=20,
                    validation_data=validation_generator,
                    callbacks=[early_stopping],
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/20
127/127 [==============================] - 106s 837ms/step - loss: 0.8827 - accuracy: 0.6729 - precision_4: 0.7732 - recall_4: 0.5593 - val_loss: 0.9901 - val_accuracy: 0.6135 - val_precision_4: 0.7548 - val_recall_4: 0.4661
Epoch 2/20
127/127 [==============================] - 111s 873ms/step - loss: 0.8359 - accuracy: 0.6882 - precision_4: 0.7890 - recall_4: 0.5800 - val_loss: 1.0115 - val_accuracy: 0.6175 - val_precision_4: 0.7340 - val_recall_4: 0.5498
Epoch 3/20
127/127 [==============================] - 106s 835ms/step - loss: 0.8215 - accuracy: 0.6882 - precision_4: 0.7846 - recall_4: 0.5884 - val_loss: 1.0064 - val_accuracy: 0.6175 - val_precision_4: 0.7273 - val_recall_4: 0.5100
Epoch 4/20
127/127 [==============================] - 106s 835ms/step - loss: 0.8308 - accuracy: 0.6853 - precision_4: 0.7873 - recall_4: 0.5781 - val_loss: 1.0198 - val_accuracy: 0.6215 - val_precision_4: 0.7228 - val_recall_4: 0.5299
Epoch 5/20
127/127 [==============================] - 10

In [24]:
model2.fit_generator(train_generator,
                    epochs=20,
                    validation_data=validation_generator,
                    callbacks=[early_stopping],
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/20
127/127 [==============================] - 106s 836ms/step - loss: 0.5534 - accuracy: 0.7994 - precision_4: 0.8550 - recall_4: 0.7485 - val_loss: 1.0899 - val_accuracy: 0.6614 - val_precision_4: 0.6916 - val_recall_4: 0.6255
Epoch 2/20
127/127 [==============================] - 108s 849ms/step - loss: 0.5433 - accuracy: 0.7969 - precision_4: 0.8576 - recall_4: 0.7559 - val_loss: 0.8589 - val_accuracy: 0.7012 - val_precision_4: 0.7321 - val_recall_4: 0.6534
Epoch 3/20
127/127 [==============================] - 106s 837ms/step - loss: 0.5486 - accuracy: 0.7959 - precision_4: 0.8377 - recall_4: 0.7500 - val_loss: 0.9199 - val_accuracy: 0.6892 - val_precision_4: 0.7630 - val_recall_4: 0.6414
Epoch 4/20
127/127 [==============================] - 106s 836ms/step - loss: 0.5271 - accuracy: 0.8019 - precision_4: 0.8552 - recall_4: 0.7648 - val_loss: 0.9496 - val_accuracy: 0.7092 - val_precision_4: 0.7489 - val_recall_4: 0.6534
Epoch 5/20
127/127 [==============================] - 10

In [25]:
model2.fit_generator(train_generator,
                    epochs=10,
                    validation_data=validation_generator,
                    callbacks=[early_stopping],
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/10
127/127 [==============================] - 106s 836ms/step - loss: 0.3476 - accuracy: 0.8745 - precision_4: 0.9004 - recall_4: 0.8533 - val_loss: 1.0279 - val_accuracy: 0.7052 - val_precision_4: 0.7350 - val_recall_4: 0.6853
Epoch 2/10
127/127 [==============================] - 106s 835ms/step - loss: 0.3578 - accuracy: 0.8725 - precision_4: 0.9048 - recall_4: 0.8503 - val_loss: 0.9023 - val_accuracy: 0.7131 - val_precision_4: 0.7558 - val_recall_4: 0.6534
Epoch 3/10
127/127 [==============================] - 106s 833ms/step - loss: 0.4224 - accuracy: 0.8473 - precision_4: 0.8781 - recall_4: 0.8295 - val_loss: 0.8807 - val_accuracy: 0.6972 - val_precision_4: 0.7124 - val_recall_4: 0.6614
Epoch 4/10
127/127 [==============================] - 106s 832ms/step - loss: 0.3853 - accuracy: 0.8557 - precision_4: 0.8882 - recall_4: 0.8281 - val_loss: 0.9634 - val_accuracy: 0.7012 - val_precision_4: 0.7434 - val_recall_4: 0.6693
Epoch 5/10
127/127 [==============================] - 10

In [34]:
model2.save('waste_classification_v2.h5')